# H3 Data Generation

In [ ]:
!pip install h3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.6/993.6 kB 12.1 MB/s eta 0:00:00


In [ ]:
import h3
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/My Drive/Class/Kepler/data/'
point_df = pd.read_csv(data_path + 'point_data.csv')

In [ ]:
point_df

,location_base,lng,lat,trip_count,passenger_count
0,DROPOFF,-73.932005,40.687939,43,59
1,DROPOFF,-74.007884,40.714960,1532,2544
2,DROPOFF,-73.953147,40.793112,310,482
3,DROPOFF,-74.008931,40.741363,1837,3212
4,DROPOFF,-73.991623,40.736546,2933,4784
...,...,...,...,...,...
8534,PICKUP,-73.843782,40.848900,1,1
8535,PICKUP,-73.822861,40.743638,1,1
8536,PICKUP,-73.876102,40.678847,1,6
8537,PICKUP,-73.788303,40.673185,1,3


In [ ]:
def add_h3_index(df, h3_resolution):
  df['h3_resolution'] = h3_resolution
  df['h3_index'] = df.apply(lambda row: h3.latlng_to_cell(row['lat'], row['lng'], h3_resolution), axis=1)
  return df

In [ ]:
h3_resolutions = [5, 6, 7]

In [ ]:
dfs = []
for res in h3_resolutions:
  df_with_h3 = add_h3_index(point_df.copy(), res)
  dfs.append(df_with_h3)

In [ ]:
combined_df = pd.concat(dfs)

In [ ]:
h3_df = combined_df.groupby(['location_base', 'h3_resolution', 'h3_index'])[['trip_count', 'passenger_count']].sum().reset_index()

In [ ]:
h3_df

,location_base,h3_resolution,h3_index,trip_count,passenger_count
0,DROPOFF,5,852a1003fffffff,1683,2924
1,DROPOFF,5,852a1007fffffff,779,1287
2,DROPOFF,5,852a100bfffffff,126255,210509
3,DROPOFF,5,852a100ffffffff,350854,582210
4,DROPOFF,5,852a1013fffffff,72,107
...,...,...,...,...,...
1407,PICKUP,7,872aa845affffff,2,2
1408,PICKUP,7,872aaca14ffffff,1,3
1409,PICKUP,7,872aadcb2ffffff,1,1
1410,PICKUP,7,872ac6140ffffff,1,4


In [ ]:
h3_df.to_csv('/content/drive/My Drive/Class/Kepler/data/h3_data.csv', index=False, encoding='utf-8')